In [ ]:
import numpy as np
import time

In [ ]:
#consider these as global variables
# set it to 3 for 8-puzzle and 4 for 15-puzzle
puzzleDimen = 4

In [ ]:
# Function to convert number into string
# Switcher is dictionary data type here
def numbers_to_strings(argument):
    switcher = {
        0: "zero",
        1: "one",
        2: "two",
    }
 
    # get() method of dictionary data type returns
    # value of passed argument if it is present
    # in dictionary otherwise second argument will
    # be assigned as default value of passed argument
    return switcher.get(argument, "nothing")
 
# Driver program
if __name__ == "__main__":
    argument=2
    print (numbers_to_strings(argument))

two


In [ ]:
class Node(): # Deals with lists to convert them into nodes
  def __init__(self, state, parent, action, depth, step_cost, path_cost, heuristic_cost):
      self.state = state # Attribute of the node being it will be a state
      self.parent = parent # This will be the parent node
      self.action = action # This will be responsible for the movement of the node
      self.depth = depth # This will help out with dealing with the depth the node is on
      self.step_cost = step_cost # Reprensents g(n), the cost of each step taken
      self.path_cost = path_cost # This is the accumulated cost of g(n), the cost to reach the current node
      self.heuristic_cost = heuristic_cost

      # The child nodes
      self.move_up = None
      self.move_down = None
      self.move_left = None
      self.move_right = None

  def try_up(self):
      # This is the index of the empty tile
      zero_index=[i[0] for i in np.where(self.state==0)]
      if zero_index[0] == puzzleDimen-1:
      #if zero_index[0] == 2:
        return False
      else:
        lower_val = self.state[zero_index[0]+1,zero_index[1]] # This gets the value of the upper title
        new_state = self.state.copy()
        new_state[zero_index[0], zero_index[1]] = lower_val # The above tile becomes the up value
        new_state[zero_index[0]+1, zero_index[1]] = 0 # The tile below the up value BECOMES the previous value
        return new_state, lower_val
  

  # This will see if a move is valid or not
  def try_down(self):
    # This is the index of the empty tile
    #zero_index=[i[0] for i in np.where(self.state==0)]
    zero_index=[i[0] for i in np.where(self.state==0)]
    if zero_index[0] == 0:
      return False
    else:
      up_val = self.state[zero_index[0]-1,zero_index[1]] # This gets the value of the upper title
      new_state = self.state.copy()
      new_state[zero_index[0], zero_index[1]] = up_val # The above tile becomes the up value
      new_state[zero_index[0]-1, zero_index[1]] = 0 # The tile below the up value BECOMES the previous value
      return new_state, up_val

    # This will see if a move is valid or not
  def try_left(self):
    # This is the index of the empty tile
    zero_index=[i[0] for i in np.where(self.state==0)]
    if zero_index[1] == puzzleDimen-1:
    #if zero_index[1] == 2:
       return False
    else:
      right_val = self.state[zero_index[0],zero_index[1]+1] # This gets the value of the upper title
      new_state = self.state.copy()
      new_state[zero_index[0], zero_index[1]] = right_val # The above tile becomes the up value
      new_state[zero_index[0], zero_index[1]+1] = 0 # The tile below the up value BECOMES the previous value
      return new_state, right_val

  # This will see if a move is valid or not
  def try_right(self):
      # This is the index of the empty tile
    zero_index=[i[0] for i in np.where(self.state==0)]
    if zero_index[1] == 0:
      return False
    else:
      left_val = self.state[zero_index[0],zero_index[1]-1] # This gets the value of the upper title
      new_state = self.state.copy()
      new_state[zero_index[0], zero_index[1]] = left_val # The above tile becomes the up value
      new_state[zero_index[0], zero_index[1]-1] = 0 # The tile below the up value BECOMES the previous value
      return new_state, left_val

    # This will get the user specific heuristic cost
  def grab_h_cost(self, new_state, goal_state, heuristic_function,path_cost, depth):
    if heuristic_function == 'new_misplaced':
      return self.h_manhattan_cost(new_state, goal_state)
    elif heuristic_function == 'manhattan':
      return self.h_manhattan_cost(new_state, goal_state)
    elif heuristic_function == 'fair_manhattan':
       return self.h_manhattan_cost(new_state, goal_state) - path_cost + depth

    # This will get the misplacement values
  def h_misplace_cost(set, new_state, goal_state):
    cost = np.sum(new_state != goal_state)-1 # The minus 1 is to exclude the empty title out of the calculation
    if cost > 0:
      return cost
    else: #This is when all the tiles match
      return 0 

  def h_manhattan_cost(self,new_state, goal_state):
     current = new_state
      # digital and coordiantes they're supposed to be
     goal_position_dic = {1:(0,0),2:(0,1),3:(0,2),4:(1,0),5:(1,1),6:(1,2),7:(2,0),8:(2,1),0:(2,2)}
     #goal_position_dic = {1:(0,0),2:(0,1),3:(0,2),4:(0,3),5:(1,0),6:(1,1),7:(1,2),8:(1,3),9:(2,0),10:(2,1),11:(2,2),12:(2,3),13:(3,0),14:(3,1),15:(3,2),0:(3,3)}
     sum_manhattan = 0 # This is the starting value
     for i in range(3):
       for j in range(3):
         if current[i,j] != 0:
           sum_manhattan += sum(abs(a-b) for a,b in zip((i,j), goal_position_dic[current[i,j]]))
     return sum_manhattan

  def movements(self): # This will show the complete movement path as a string
     state_temp_trace = [self.state]
     action_path_trace = [self.action]
     show_action = ''
     peek = ''
     
     while self.parent:
       self = self.parent
       action_path_trace.append(self.action)

     while action_path_trace:
       peek = action_path_trace.pop()

       if (peek == None):
         peek = ''
       else:
         show_action = show_action + peek

     print('Actions Taken:', str(show_action))
       
  # This will trace back all the way to the root node printing all the data needed
  def print_path(self):
      # Make a FIFO stack to place the trace
     state_trace = [self.state]
     action_trace = [self.action]
     depth_trace = [self.depth]
     step_cost_trace = [self.step_cost]
     path_cost_trace = [self.path_cost]
     heuristic_cost_trace = [self.heuristic_cost]
     
    # add the node information tracing up the tree
     while self.parent:
        self = self.parent

        state_trace.append(self.state)
        action_trace.append(self.action)
        depth_trace.append(self.depth)
        step_cost_trace.append(self.step_cost)
        path_cost_trace.append(self.path_cost)
        heuristic_cost_trace.append(self.heuristic_cost)
      
      # This will print out the path
     steps = 0
     while state_trace:
       print('step',steps)
       print(state_trace.pop())
       print('action:',action_trace.pop(),'| depth=', str(depth_trace.pop()),'| step cost=', str(step_cost_trace.pop()),'| total cost=',
             str(path_cost_trace.pop() + heuristic_cost_trace.pop()) )
       print()

       steps += 1

  def BFS(self, goal_state):
    start = time.time() # To see the time complexity

    queue = [self] # queue of found but unvisited nodes
    queue_num_nodes_popped = 0 # number of nodes popped off the queue
    queue_max_length = 1 # max number of nodes in the queue, measuring space preformance

    depth_queue = [0] # queue of depth nodes
    path_cost_queue = [0] # queue cost of path for nodes
    visited = set([]) # This will recorded the nodes visited which starts out empty

    while queue:
      #update maximum length in the queue
      if len(queue) > queue_max_length:
        queue_max_length = len(queue)

      current_node = queue.pop(0) #select and pop first node in the queue
      queue_num_nodes_popped += 1

      current_depth = depth_queue.pop(0) #select and remove depth from current node
      current_path_cost = path_cost_queue.pop(0) # select and remove path cost from reaching current node
      visited.add(tuple(current_node.state.reshape(1,puzzleDimen*puzzleDimen)[0])) # This is to advoid repeated states
      #visited.add(tuple(current_node.state(1,puzzleDimen*puzzleDimen)[0])) # This is to advoid repeated states

      # Once the goal is found, trace it back to the root node to print out
      if np.array_equal(current_node.state, goal_state):
        current_node.print_path()

        print('Nodes Visited:',len(visited))
        current_node.movements()
        print('Time performance:',str(queue_num_nodes_popped), 'nodes popped off the queue')
        print('Space performance:',str(queue_max_length), 'nodes in the queue at its max')
        print('Time spent: %0.2fs' % (time.time()-start))
        return True

      else: # This will search through the list up,down,left,right or (u,d,l,r)
        
        # see if moving lower tile up is a valid move
        if current_node.try_up():
          new_state,down_val = current_node.try_up()
          #Check if the node has already been visited
          if tuple(new_state.reshape(1,puzzleDimen*puzzleDimen)[0]) not in visited:
          #if tuple(new_state.reshape(1,9)[0]) not in visited:
            #creates a new child node
            current_node.move_up = Node(state=new_state,parent=current_node,action='d', depth=current_depth+1,
                                          step_cost=down_val,path_cost=current_path_cost+down_val,heuristic_cost=0)
            queue.append(current_node.move_up)
            depth_queue.append(current_depth+1)
            path_cost_queue.append(current_path_cost+down_val)
        

        # see if moving upper tile down is a valid move
        if current_node.try_down():
          new_state,up_val = current_node.try_down()
          #Check if the node has already been visited
          if tuple(new_state.reshape(1,puzzleDimen*puzzleDimen)[0]) not in visited:
          #if tuple(new_state.reshape(1,9)[0]) not in visited:
            #creates a new child node
            current_node.move_down = Node(state=new_state,parent=current_node,action='u', depth=current_depth+1,
                                          step_cost=up_val,path_cost=current_path_cost+up_val,heuristic_cost=0)
            queue.append(current_node.move_down)
            depth_queue.append(current_depth+1)
            path_cost_queue.append(current_path_cost+up_val)

         # see if moving right tile left is a valid move
        if current_node.try_left():
          new_state,right_val = current_node.try_left()
          #Check if the node has already been visited
          if tuple(new_state.reshape(1,puzzleDimen*puzzleDimen)[0]) not in visited:
          #if tuple(new_state.reshape(1,9)[0]) not in visited:
            #creates a new child node
            current_node.move_left = Node(state=new_state,parent=current_node,action='r', depth=current_depth+1,
                                          step_cost=right_val, path_cost=current_path_cost+right_val, heuristic_cost=0)
            queue.append(current_node.move_left)
            depth_queue.append(current_depth+1)
            path_cost_queue.append(current_path_cost+right_val)


        # see if moving left tile right is a valid move
        if current_node.try_right():
          new_state,left_val = current_node.try_right()
          #Check if the node has already been visited
          if tuple(new_state.reshape(1,puzzleDimen*puzzleDimen)[0]) not in visited:
          #if tuple(new_state.reshape(1,9)[0]) not in visited:
            #creates a new child node
            current_node.move_right = Node(state=new_state,parent=current_node,action='l', depth=current_depth+1,
                                          step_cost=left_val,path_cost=current_path_cost+left_val,heuristic_cost=0)
            queue.append(current_node.move_right)
            depth_queue.append(current_depth+1)
            path_cost_queue.append(current_path_cost+left_val)

#==========================================================================================================================

  def DFS(self, goal_state):
    start = time.time() # To see the time complexity

    queue = [self] # queue of found but unvisited nodes
    queue_num_nodes_popped = 0 # number of nodes popped off the queue
    queue_max_length = 1 # max number of nodes in the queue, measuring space preformance

    depth_queue = [0] # queue of depth nodes
    path_cost_queue = [0] # queue cost of path for nodes
    visited = set([]) # This will recorded the nodes visited which starts out empty

    while queue:
      #update maximum length in the queue
      if len(queue) > queue_max_length:
        queue_max_length = len(queue)

      current_node = queue.pop(0) #select and pop first node in the queue
      queue_num_nodes_popped += 1

      current_depth = depth_queue.pop(0) #select and remove depth from current node
      current_path_cost = path_cost_queue.pop(0) # select and remove path cost from reaching current node
      #visited.add(tuple(current_node.state.reshape(1,16)[0])) # This is to advoid repeated states
      visited.add(tuple(current_node.state.reshape(1,puzzleDimen*puzzleDimen)[0])) # This is to advoid repeated states

      # Once the goal is found, trace it back to the root node to print out
      if np.array_equal(current_node.state, goal_state):
        current_node.print_path()

        print('Nodes Visited:',len(visited))
        current_node.movements()
        print('Time performance:',str(queue_num_nodes_popped), 'nodes popped off the queue')
        print('Space performance:',str(queue_max_length), 'nodes in the queue at its max')
        print('Time spent: %0.2fs' % (time.time()-start))
        return True

      else: # This will search through the list up,down,left,right or (u,d,l,r)
        
        # see if moving lower tile up is a valid move
        if current_node.try_up():
          new_state,down_val = current_node.try_up()
          #Check if the node has already been visited
          #if tuple(new_state.reshape(1,16)[0]) not in visited:
          if tuple(new_state.reshape(1,puzzleDimen*puzzleDimen)[0]) not in visited:
            #creates a new child node
            current_node.move_up = Node(state=new_state,parent=current_node,action='d', depth=current_depth+1,
                                          step_cost=down_val,path_cost=current_path_cost+down_val,heuristic_cost=0)
            # These insert to the begining of the queue to simulated going through the depth  
            queue.insert(0,current_node.move_up)
            depth_queue.insert(0,current_depth+1)
            path_cost_queue.insert(0,current_path_cost+down_val)
        

        # see if moving upper tile down is a valid move
        if current_node.try_down():
          new_state,up_val = current_node.try_down()
          #Check if the node has already been visited
          #if tuple(new_state.reshape(1,16)[0]) not in visited:
          if tuple(new_state.reshape(1,puzzleDimen*puzzleDimen)[0]) not in visited:
            #creates a new child node
            current_node.move_down = Node(state=new_state,parent=current_node,action='u', depth=current_depth+1,
                                          step_cost=up_val,path_cost=current_path_cost+up_val,heuristic_cost=0)
            queue.insert(0,current_node.move_down)
            depth_queue.insert(0,current_depth+1)
            path_cost_queue.insert(0,current_path_cost+up_val)

         # see if moving right tile left is a valid move
        if current_node.try_left():
          new_state,right_val = current_node.try_left()
          #Check if the node has already been visited
          #if tuple(new_state.reshape(1,16)[0]) not in visited:
          if tuple(new_state.reshape(1,puzzleDimen*puzzleDimen)[0]) not in visited:
            #creates a new child node
            current_node.move_left = Node(state=new_state,parent=current_node,action='r', depth=current_depth+1,
                                          step_cost=right_val, path_cost=current_path_cost+right_val, heuristic_cost=0)
            queue.insert(0,current_node.move_left)
            depth_queue.insert(0,current_depth+1)
            path_cost_queue.insert(0,current_path_cost+right_val)


        # see if moving left tile right is a valid move
        if current_node.try_right():
          new_state,left_val = current_node.try_right()
          #Check if the node has already been visited
          #if tuple(new_state.reshape(1,16)[0]) not in visited:
          if tuple(new_state.reshape(1,puzzleDimen*puzzleDimen)[0]) not in visited:
            #creates a new child node
            current_node.move_right = Node(state=new_state,parent=current_node,action='l', depth=current_depth+1,
                                          step_cost=left_val,path_cost=current_path_cost+left_val,heuristic_cost=0)
            queue.insert(0,current_node.move_right)
            depth_queue.insert(0,current_depth+1)
            path_cost_queue.insert(0,current_path_cost+left_val)

#==================================================================================================================

#==========================================================================================================================

  def IterativeDFS(self, goal_state):
    start = time.time() # To see the time complexity

    queue_num_nodes_popped = 0 # number of nodes popped off the queue
    queue_max_length = 1 # max number of nodes in the queue, measuring space preformance

    # This will be the limit on the depth of the tree it can search through that is (120)
    for depth_limit in range(120):


      queue = [self] # queue of found but unvisited nodes
      depth_queue = [0] # queue of depth nodes
      path_cost_queue = [0] # queue cost of path for nodes
      visited = set([]) # This will recorded the nodes visited which starts out empty

      while queue:
        #update maximum length in the queue
        if len(queue) > queue_max_length:
          queue_max_length = len(queue)

        current_node = queue.pop(0) #select and pop first node in the queue
        queue_num_nodes_popped += 1

        current_depth = depth_queue.pop(0) #select and remove depth from current node
        current_path_cost = path_cost_queue.pop(0) # select and remove path cost from reaching current node
        #visited.add(tuple(current_node.state.reshape(1,16)[0])) # This is to advoid repeated states
        visited.add(tuple(current_node.state.reshape(1,puzzleDimen*puzzleDimen)[0])) # This is to advoid repeated states

        # Once the goal is found, trace it back to the root node to print out
        if np.array_equal(current_node.state, goal_state):
          current_node.print_path()

          print('Nodes Visited:',len(visited))
          current_node.movements()
          print('Time performance:',str(queue_num_nodes_popped), 'nodes popped off the queue')
          print('Space performance:',str(queue_max_length), 'nodes in the queue at its max')
          print('Time spent: %0.2fs' % (time.time()-start))
          return True

        else: # This will search through the list up,down,left,right or (u,d,l,r)

          if current_depth < depth_limit:
            
            # see if moving lower tile up is a valid move
            if current_node.try_up():
              new_state,down_val = current_node.try_up()
              #Check if the node has already been visited
              #if tuple(new_state.reshape(1,16)[0]) not in visited:
              if tuple(new_state.reshape(1,puzzleDimen*puzzleDimen)[0]) not in visited:
                #creates a new child node
                current_node.move_up = Node(state=new_state,parent=current_node,action='d', depth=current_depth+1,
                                              step_cost=down_val,path_cost=current_path_cost+down_val,heuristic_cost=0)
                # These insert to the begining of the queue to simulated going through the depth  
                queue.insert(0,current_node.move_up)
                depth_queue.insert(0,current_depth+1)
                path_cost_queue.insert(0,current_path_cost+down_val)
            

            # see if moving upper tile down is a valid move
            if current_node.try_down():
              new_state,up_val = current_node.try_down()
              #Check if the node has already been visited
              #if tuple(new_state.reshape(1,16)[0]) not in visited:
              if tuple(new_state.reshape(1,puzzleDimen*puzzleDimen)[0]) not in visited:
                #creates a new child node
                current_node.move_down = Node(state=new_state,parent=current_node,action='u', depth=current_depth+1,
                                              step_cost=up_val,path_cost=current_path_cost+up_val,heuristic_cost=0)
                queue.insert(0,current_node.move_down)
                depth_queue.insert(0,current_depth+1)
                path_cost_queue.insert(0,current_path_cost+up_val)

            # see if moving right tile left is a valid move
            if current_node.try_left():
              new_state,right_val = current_node.try_left()
              #Check if the node has already been visited
              #if tuple(new_state.reshape(1,16)[0]) not in visited:
              if tuple(new_state.reshape(1,puzzleDimen*puzzleDimen)[0]) not in visited:
                #creates a new child node
                current_node.move_left = Node(state=new_state,parent=current_node,action='r', depth=current_depth+1,
                                              step_cost=right_val, path_cost=current_path_cost+right_val, heuristic_cost=0)
                queue.insert(0,current_node.move_left)
                depth_queue.insert(0,current_depth+1)
                path_cost_queue.insert(0,current_path_cost+right_val)


            # see if moving left tile right is a valid move
            if current_node.try_right():
              new_state,left_val = current_node.try_right()
              #Check if the node has already been visited
              #if tuple(new_state.reshape(1,16)[0]) not in visited:
              if tuple(new_state.reshape(1,puzzleDimen*puzzleDimen)[0]) not in visited:
                #creates a new child node
                current_node.move_right = Node(state=new_state,parent=current_node,action='l', depth=current_depth+1,
                                              step_cost=left_val,path_cost=current_path_cost+left_val,heuristic_cost=0)
                queue.insert(0,current_node.move_right)
                depth_queue.insert(0,current_depth+1)
                path_cost_queue.insert(0,current_path_cost+left_val)

#==================================================================================================================

#==========================================================================================================================

  def UCS(self, goal_state):
    start = time.time() # To see the time complexity

    queue = [(self,0)] # queue of found but unvisited nodes with cost
    queue_num_nodes_popped = 0 # number of nodes popped off the queue
    queue_max_length = 1 # max number of nodes in the queue, measuring space preformance

    depth_queue = [(0,0)] # queue of depth nodes
    path_cost_queue = [0] # queue cost of path for nodes
    visited = set([]) # This will recorded the nodes visited which starts out empty

    while queue:
      #This will calculate the cost
      queue = sorted(queue, key=lambda x: x[1])
      depth_queue = sorted(depth_queue, key=lambda x: x[1])
      path_cost_queue = sorted(path_cost_queue, key=lambda x: x)


      #update maximum length in the queue
      if len(queue) > queue_max_length:
        queue_max_length = len(queue)

      current_node = queue.pop(0)[0] #select and pop first node in the queue
      queue_num_nodes_popped += 1

      current_depth = depth_queue.pop(0)[0] #select and remove depth from current node
      current_path_cost = path_cost_queue.pop(0) # select and remove path cost from reaching current node
      visited.add(tuple(current_node.state.reshape(1,puzzleDimen*puzzleDimen)[0])) # This is to advoid repeated states
      #visited.add(tuple(current_node.state.reshape(1,9)[0])) # This is to advoid repeated states

      # Once the goal is found, trace it back to the root node to print out
      if np.array_equal(current_node.state, goal_state):
        current_node.print_path()

        print('Nodes Visited:',len(visited))
        current_node.movements()
        print('Time performance:',str(queue_num_nodes_popped), 'nodes popped off the queue')
        print('Space performance:',str(queue_max_length), 'nodes in the queue at its max')
        print('Time spent: %0.2fs' % (time.time()-start))
        return True

      else: # This will search through the list up,down,left,right or (u,d,l,r)
        
        # see if moving lower tile up is a valid move
        if current_node.try_up():
          new_state,down_val = current_node.try_up()
          #Check if the node has already been visited
          if tuple(new_state.reshape(1,puzzleDimen*puzzleDimen)[0]) not in visited:
          #if tuple(new_state.reshape(1,9)[0]) not in visited:
            #creates a new child node
            current_node.move_up = Node(state=new_state,parent=current_node,action='d', depth=current_depth+1,
                                              step_cost=down_val,path_cost=current_path_cost+down_val,heuristic_cost=0)
            # These insert to the begining of the queue to simulated going through the depth  
            queue.append((current_node.move_up, current_path_cost+down_val))
            depth_queue.append((current_depth+1, current_path_cost+down_val))
            path_cost_queue.append(current_path_cost+down_val)
        

        # see if moving upper tile down is a valid move
        if current_node.try_down():
          new_state,up_val = current_node.try_down()
          #Check if the node has already been visited
          if tuple(new_state.reshape(1,puzzleDimen*puzzleDimen)[0]) not in visited:
          #if tuple(new_state.reshape(1,9)[0]) not in visited:
            #creates a new child node
            current_node.move_down = Node(state=new_state,parent=current_node,action='u', depth=current_depth+1,
                                          step_cost=up_val,path_cost=current_path_cost+up_val,heuristic_cost=0)
            # These insert to the begining of the queue to simulated going through the depth  
            queue.append((current_node.move_down, current_path_cost+up_val))
            depth_queue.append((current_depth+1, current_path_cost+up_val))
            path_cost_queue.append(current_path_cost+up_val)
        

         # see if moving right tile left is a valid move
        if current_node.try_left():
          new_state,right_val = current_node.try_left()
          #Check if the node has already been visited
          if tuple(new_state.reshape(1,puzzleDimen*puzzleDimen)[0]) not in visited:
          #if tuple(new_state.reshape(1,9)[0]) not in visited:
            #creates a new child node
            current_node.move_left = Node(state=new_state,parent=current_node,action='r', depth=current_depth+1,
                                          step_cost=right_val, path_cost=current_path_cost+right_val, heuristic_cost=0)
             # These insert to the begining of the queue to simulated going through the depth  
            queue.append((current_node.move_right, current_path_cost+right_val))
            depth_queue.append((current_depth+1, current_path_cost+right_val))
            path_cost_queue.append(current_path_cost+right_val)
        


        # see if moving left tile right is a valid move
        if current_node.try_right():
          new_state,left_val = current_node.try_right()
          #Check if the node has already been visited
          if tuple(new_state.reshape(1,puzzleDimen*puzzleDimen)[0]) not in visited:
          #if tuple(new_state.reshape(1,9)[0]) not in visited:
            #creates a new child node
            current_node.move_right = Node(state=new_state,parent=current_node,action='l', depth=current_depth+1,
                                          step_cost=left_val,path_cost=current_path_cost+left_val,heuristic_cost=0)
            # These insert to the begining of the queue to simulated going through the depth  
            queue.append((current_node.move_up, current_path_cost+left_val))
            depth_queue.append((current_depth+1, current_path_cost+left_val))
            path_cost_queue.append(current_path_cost+left_val)
        

#==================================================================================================================
#==========================================================================================================================

  def AStarManhattan(self, goal_state, heuristic_function):
    start = time.time() # To see the time complexity

    queue = [(self,0)] # queue of found but unvisited nodes with cost
    queue_num_nodes_popped = 0 # number of nodes popped off the queue
    queue_max_length = 1 # max number of nodes in the queue, measuring space preformance

    depth_queue = [(0,0)] # queue of depth nodes
    path_cost_queue = [(0,0)] # queue cost of path for nodes
    visited = set([]) # This will recorded the nodes visited which starts out empty

    while queue:
      #This will calculate the cost
      queue = sorted(queue, key=lambda x: x[1])
      depth_queue = sorted(depth_queue, key=lambda x: x[1])
      path_cost_queue = sorted(path_cost_queue, key=lambda x: x)


      #update maximum length in the queue
      if len(queue) > queue_max_length:
        queue_max_length = len(queue)

      current_node = queue.pop(0)[0] #select and pop first node in the queue
      queue_num_nodes_popped += 1

      current_depth = depth_queue.pop(0) #select and remove depth from current node
      current_path_cost = path_cost_queue.pop(0) # select and remove path cost from reaching current node
      visited.add(tuple(current_node.state.reshape(1,puzzleDimen*puzzleDimen)[0])) # This is to advoid repeated states
      #visited.add(tuple(current_node.state.reshape(1,9)[0])) # This is to advoid repeated states

      # Once the goal is found, trace it back to the root node to print out
      if np.array_equal(current_node.state, goal_state):
        current_node.print_path()

        print('Nodes Visited:',len(visited))
        current_node.movements()
        print('Time performance:',str(queue_num_nodes_popped), 'nodes popped off the queue')
        print('Space performance:',str(queue_max_length), 'nodes in the queue at its max')
        print('Time spent: %0.2fs' % (time.time()-start))
        return True

      else: # This will search through the list up,down,left,right or (u,d,l,r)
        
        # see if moving lower tile up is a valid move
        if current_node.try_up():
          new_state,down_val = current_node.try_up()
          #Check if the node has already been visited
          if tuple(new_state.reshape(1,puzzleDimen*puzzleDimen)[0]) not in visited:
          #if tuple(new_state.reshape(1,9)[0]) not in visited:
            #Path cost is this plus Whatever down value it has
            path_cost=current_path_cost+down_val
            #Heuristic value
            h_cost = self.grab_h_cost(new_state,goal_state,heuristic_function,path_cost,current_depth)
            #creates a new child node
            total_cost = path_cost+h_cost
            current_node.move_up = Node(state=new_state,parent=current_node,action='d', depth=current_depth+1,
                                          step_cost=down_val,path_cost=path_cost,heuristic_cost=h_cost) #You will have to copy these lines for the next ones
            # These insert to the begining of the queue to simulated going through the depth  
            queue.append((current_node.move_up, total_cost))
            depth_queue.append((current_depth+1, total_cost))
            path_cost_queue.append((path_cost, total_cost))
        

        # see if moving upper tile down is a valid move
        if current_node.try_down():
          new_state,up_val = current_node.try_down()
          #Check if the node has already been visited
          if tuple(new_state.reshape(1,puzzleDimen*puzzleDimen)[0]) not in visited:
          #if tuple(new_state.reshape(1,9)[0]) not in visited:
             #Path cost is this plus Whatever down value it has
            path_cost=current_path_cost+up_val
            #Heuristic value
            h_cost = self.grab_h_cost(new_state,goal_state,heuristic_function,path_cost,current_depth)
            #creates a new child node
            total_cost = path_cost+h_cost
            #creates a new child node
            current_node.move_down = Node(state=new_state,parent=current_node,action='u', depth=current_depth+1,
                                          step_cost=up_val,path_cost=path_cost,heuristic_cost=h_cost)
            # These insert to the begining of the queue to simulated going through the depth  
            queue.append((current_node.move_down, total_cost))
            depth_queue.append((current_depth+1, total_cost))
            path_cost_queue.append((path_cost, total_cost))
        

         # see if moving right tile left is a valid move
        if current_node.try_left():
          new_state,right_val = current_node.try_left()
          #Check if the node has already been visited
          if tuple(new_state.reshape(1,puzzleDimen*puzzleDimen)[0]) not in visited:
          #if tuple(new_state.reshape(1,9)[0]) not in visited:
             #Path cost is this plus Whatever down value it has
            path_cost=current_path_cost+right_val
            #Heuristic value
            h_cost = self.grab_h_cost(new_state,goal_state,heuristic_function,path_cost,current_depth)
            #creates a new child node
            total_cost = path_cost+h_cost
            #creates a new child node
            current_node.move_left = Node(state=new_state,parent=current_node,action='r', depth=current_depth+1,
                                          step_cost=right_val, path_cost=path_cost, heuristic_cost=h_cost)
             # These insert to the begining of the queue to simulated going through the depth  
            queue.append((current_node.move_left, total_cost))
            depth_queue.append((current_depth+1, total_cost))
            path_cost_queue.append((path_cost, total_cost))
        


        # see if moving left tile right is a valid move
        if current_node.try_right():
          new_state,left_val = current_node.try_right()
          #Check if the node has already been visited
          if tuple(new_state.reshape(1,puzzleDimen*puzzleDimen)[0]) not in visited:
          #if tuple(new_state.reshape(1,9)[0]) not in visited:
             #Path cost is this plus Whatever down value it has
            path_cost=current_path_cost+left_val
            #Heuristic value
            h_cost = self.grab_h_cost(new_state,goal_state,heuristic_function,path_cost,current_depth)
            #creates a new child node
            total_cost = path_cost+h_cost
            #creates a new child node
            current_node.move_right = Node(state=new_state,parent=current_node,action='l', depth=current_depth+1,
                                          step_cost=left_val,path_cost=path_cost,heuristic_cost=h_cost)
            # These insert to the begining of the queue to simulated going through the depth  
            queue.append((current_node.move_right, total_cost))
            depth_queue.append((current_depth+1, total_cost))
            path_cost_queue.append((path_cost, total_cost))
        

#==================================================================================================================


In [ ]:
# This is for testing out the functions
test = np.array([1,2,3,8,6,4,7,5,0]).reshape(3,3)
easy = np.array([1,2,3,4,0,5,7,8,6]).reshape(3,3)
easy2 = "123406578"
medium = np.array([2,8,1,0,4,3,7,6,5]).reshape(3,3)
hard = np.array([5,6,7,4,0,8,3,2,1]).reshape(3,3)

problem1 = np.array([1,6,0,2,7,3,4,8,5]).reshape(3,3)
problem2 = np.array([4,6,2,3,0,1,5,8,7]).reshape(3,3)
problem3 = np.array([8,2,1,5,7,4,3,6,0]).reshape(3,3)
problem4 = np.array([8,4,0,1,5,6,3,7,2]).reshape(3,3)
problem5 = np.array([5,3,0,4,7,8,1,2,6]).reshape(3,3)
problem6 = np.array([0,6,8,3,1,4,2,5,7]).reshape(3,3)
problem7 = np.array([4,5,3,2,0,7,1,8,6]).reshape(3,3)
problem8 = np.array([1,2,8,3,0,7,6,4,5]).reshape(3,3)
problem9 = np.array([0,3,5,6,8,4,7,1,2]).reshape(3,3)
problem10 = np.array([6,8,4,3,1,7,0,2,5]).reshape(3,3)
problem11 = np.array([0,2,8,5,1,4,6,3,7]).reshape(3,3)
problem12 = np.array([6,1,8,2,7,3,5,4,0]).reshape(3,3)
problem13 = np.array([0,4,2,3,8,5,6,7,1]).reshape(3,3)
problem14 = np.array([4,2,0,3,8,5,7,1,6]).reshape(3,3)
problem15 = np.array([0,5,4,6,7,2,8,1,3]).reshape(3,3)
problem16 = np.array([3,1,4,5,7,2,6,8,0]).reshape(3,3)
problem17 = np.array([6,3,7,2,1,8,0,4,5]).reshape(3,3)
problem18 = np.array([4,3,0,6,2,1,8,7,5]).reshape(3,3)
problem19 = np.array([1,5,8,2,7,4,0,3,6]).reshape(3,3)
problem20 = np.array([1,3,0,4,5,8,7,2,6]).reshape(3,3)

A = 10
B = 11
C = 12
D = 13
E = 14
F = 15

# This is for the 15 puzzle
# 10 = A, 11 = B, 12 = C, 13 = D, 14 = E, 15 = F
easy15P1 = np.array([1,6,2,3,5,10,7,4,9,12,0,8,13,14,11,15]).reshape(4,4)
easy15P2 = np.array([0,6,3,4,2,1,7,8,5,9,A,B,D,E,F,C]).reshape(4,4)
easy15P3 = np.array([0,1,2,4,5,6,3,7,9,B,C,8,D,A,E,F]).reshape(4,4)
easy15P4 = np.array([5,1,2,4,6,A,3,8,0,9,7,B,D,E,F,C]).reshape(4,4)
easy15P5 = np.array([1,2,3,4,5,6,7,8,D,9,C,F,E,B,A,0]).reshape(4,4)

Hard15P1 = np.array([7,1,A,9,2,C,E,0,3,D,B,4,6,5,8,F]).reshape(4,4)
Hard15P2 = np.array([0,2,3,4,8,6,9,7,D,F,5,A,1,E,B,C]).reshape(4,4)
Hard15P3 = np.array([3,9,A,1,D,0,E,C,7,B,F,8,6,4,5,2]).reshape(4,4)
Hard15P4 = np.array([E,A,B,4,8,0,F,C,1,9,D,5,6,2,3,7]).reshape(4,4)
Hard15P5 = np.array([7,D,B,1,3,C,5,2,F,4,6,E,8,0,A,9]).reshape(4,4)



# This is for the 15 puzzle
# 10 = A, 11 = B, 12 = C, 13 = D, 14 = E, 15 = F
easy15 = np.array([1,2,3,8,4,10,7,6,5,0,9,11,12,13,14,15]).reshape(4,4)
hard15 = np.array([10,2,3,14,15,0,6,9,8,7,1,11,12,13,4,5]).reshape(4,4)

initial_state = Hard15P3
#goal_state = "123456780"
#goal_state = np.array([1,2,3,4,5,6,7,8,0]).reshape(3,3)
goal_state = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0]).reshape(4,4)
print(initial_state, '\n')
print(goal_state)

[[ 3  9 10  1]
 [13  0 14 12]
 [ 7 11 15  8]
 [ 6  4  5  2]] 

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15  0]]


In [ ]:
# The root node to start all this none sense
root_node = Node(state=initial_state,action=None,parent=None,depth=0,step_cost=0,path_cost=0,heuristic_cost=0)

In [ ]:
# Search Level by Level with queue
root_node.BFS(goal_state)

KeyboardInterrupt: ignored

In [ ]:
# This is the normal depth-first search 
# (Warning: It works but it takes a looooong time to solve)
root_node.DFS(goal_state)

KeyboardInterrupt: ignored

In [ ]:
#This is for the iterative depth-first search
root_node.IterativeDFS(goal_state)

In [ ]:
root_node.UCS(goal_state)

AttributeError: ignored

In [ ]:
root_node.AStarManhattan(goal_state, heuristic_function = 'manhattan')

TypeError: ignored